# Projet 3 - Stats

In [1]:
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

import utils
import project

In [3]:
cerevisae = utils.read_fasta('sequences/yeast_s_cerevisae_genomic_chr1-4.fna')
len(cerevisae)

4

In [4]:
chromos = [seq for seq in cerevisae.values()]
chromos_flattened = [el for sub in chromos for el in sub]
freqs = utils.nucleotide_count(chromos_flattened)
probas = utils.nucleotide_frequency(chromos_flattened)

In [5]:
project.logproba(chromos_flattened, probas)

-3438164.462646954

In [6]:
project.logprobafast(freqs)

-3438164.4626394957

In [7]:
freqs

[754342, 505943, 503163, 752405]

In [8]:
project.logproba([1, 0, 3], (0.2, 0.3, 0.1, 0.4))

-3.7297014486341915

In [9]:
project.code('TAC', 3)

49

In [10]:
project.inv_code(49, 3)

'TAC'

In [11]:
project.inv_nucleotide

{0: 'A', 1: 'C', 2: 'G', 3: 'T'}

In [12]:
project.int_to_str('301')

'TAC'

In [13]:
project.n_grams('ATCAT', 2)

{'AT': 2, 'TC': 1, 'CA': 1}